In [ ]:
import pandas as pd
import numpy as np
from bff_processor.Systogram import Systogram
from bff_processor.bff_meta import *
from bff_processor.sig_op_func import *
from bff_processor.bff_cuts import bff_1, bff_2, reg_filter
from bff_processor.plotting_utils import unc_plot, boost2unc
from bff_processor.utils import nratio_plot_template, hist2unc, vunc2nom, chiSquared
import os
import re
from time import perf_counter
import boost_histogram as bh
import pyarrow.feather as feather
import zfit
from zfit import z
from math import pi
import mplhep as hep
hep.style.use(hep.style.CMS)
plt.rcParams.update({
    "text.usetex": True,
})
from zfit.models.physics import DoubleCB

In [ ]:
import pickle

In [ ]:
from zfit.models.physics import double_crystalball_func
class DoubleCBError(zfit.pdf.ZPDF):
    _PARAMS = ['mu', 'sigma',
               'alphal', 'alphar',
              'nl', 'nr']
        
    def uncertainty(self, x):
        import uncertainties.unumpy as unp
        from uncertainties import ufloat
        y = self.pdf(x) 
        return np.asarray([ufloat(yval,0) for yval in y])  
    
    def _unnormalized_pdf(self, x):  # implement function
        data = z.unstack_x(x)
        mu = self.params['mu']
        sigma = self.params['sigma']
        alphal = self.params['alphal']
        alphar = self.params['alphar']
        nl = self.params['nl']
        nr = self.params['nr']
        return double_crystalball_func(data, mu=mu, sigma=sigma, alphal=alphal, alphar=alphar, nl=nl, nr=nr)
    
    def par2ufloat(self, name): 
        assert self.result, "Need to add result object"
        from uncertainties import ufloat
        return  ufloat(self.result.params[name]['value'], self.result.params[name]['minuit_hesse']['error'])

    def uncertainty(self, x):
        import uncertainties.unumpy as unp
        data = x
        name = []
        values = []
        for x in self.result.params:
            name.append(x.name)
            values.append(x.value().numpy())
        print(values)
        print(self.result.covariance())
        import uncertainties
        popt = uncertainties.correlated_values(values, self.result.covariance())
        
        y = self.lognorm(data, unp, **{n:p for n, p in zip(name,popt)}) 
        y = y * np.sum(self.pdf(data)) / np.sum(y)
        return y          
        

In [ ]:
try:
    mu = zfit.Parameter("mu", 300,  100, 600)
    sigma = zfit.Parameter("sigma_b", 20,  0, 100)
    alphal = zfit.Parameter("alphal", 2,  0, 100)
    nl = zfit.Parameter("nl", 3,  -10, 10)
    alphar = zfit.Parameter("alphar", 40,  0, 100)
    nr = zfit.Parameter("nr", 10,  -100, 100)
    Nsig = zfit.Parameter("Nsig", 1., -20., 1e8)
except:
    print("already defined")

In [ ]:
obs = zfit.Space("x", limits=[110,800])
doublecb = DoubleCBError(obs=obs, mu=mu, sigma=sigma,
                         alphal=alphal, nl=nl, 
                         alphar=alphar, nr=nr)


In [ ]:
era = 2016
df = feather.read_feather('data/combined_{}.feather'.format(era))

In [ ]:
bff_data = df[df.name.str.contains("BFF")]
df  = bff_data[(bff_data.mass>130) & (bff_data.mass < 800) & (bff_data.dbs==0.04)]
df = df[df.mass == 200]
df

In [ ]:
data = df.DiLepMass.to_numpy()
weights = df.Weight.to_numpy()
mu.set_value(250)

In [ ]:
data = zfit.Data.from_numpy(obs=obs, array=data, weights=weights)

In [ ]:
nll = zfit.loss.UnbinnedNLL(model=doublecb, data=data)

In [ ]:
minimizer = zfit.minimize.Minuit()

In [ ]:
result = minimizer.minimize(nll)

In [ ]:
result